# TimeEmbeddingレイヤを確認する
TimeEmbeddingレイヤは、Embeddingレイヤを時間方向に結合していくレイヤである

In [1]:
import numpy as np
from common.time_layers import Embedding

In [2]:
class TimeEmbedding:
    def __init__(self, W):
        """
        W : 重み行列, word2vecの埋め込み行列に相当する。配列形状は、(語彙数、埋め込みベクトルの要素数)
        """        
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.layers = None
        self.W = W

    def forward(self, xs):
        """
        順伝播計算
        xs : 入力の単語ID, 配列形状は(バッチサイズ、時間数)
        """
        N, T = xs.shape # バッチサイズ、時間数
        V, D = self.W.shape # 語彙数、埋め込みベクトルの要素数

        # 初期化
        out = np.empty((N, T, D), dtype='f')
        self.layers = []

        # 時間方向に計算を進める
        for t in range(T):
            
            # Embeddigレイヤを生成し、順伝播計算を行う
            layer = Embedding(self.W)
            out[:, t, :] = layer.forward(xs[:, t])
            
            #  Embeddigレイヤを保持しておく
            self.layers.append(layer)

        return out

    def backward(self, dout):
        """
        逆伝播計算
        """
        N, T, D = dout.shape # バッチサイズ、時間数、埋め込みベクトルの要素数

        grad = 0
        
        # 時間方向に計算を進める(時間方向には独立しているので逆方向に進めなくてよい)
        for t in range(T):
            layer = self.layers[t]
            
            # 逆伝播計算
            layer.backward(dout[:, t, :])
            
            # 勾配を足し合わせる
            grad += layer.grads[0]

        self.grads[0][:] = grad # 同じメモリ位置に代入

        return None

In [3]:
np.random.seed(1234)

V = 10 # 語彙数
D = 3 # 埋め込みベクトルの要素数

# パラメータの初期化
embed_W = np.random.randn(V, D) 
print("embed_W=", embed_W)
print()

# オブジェクトの生成
time_emb = TimeEmbedding(embed_W)
print("id of time_emb.grads[0]", id(time_emb.grads[0]))
print()

# バッチサイズ
N = 2

# 時間数
T = 4

# 単語ID
xs = np.array([[2, 5, 1, 4],[3, 2, 7, 8]])

# 順伝播計算
time_emb.forward(xs)

# 逆伝播計算
dout = np.random.randn(N, T, D)
# print("dout=", dout)
# print()
time_emb.backward(dout)
# print("dW=", time_emb.grads[0])
# print()

print("id of time_emb.grads[0]", id(time_emb.grads[0]))
print()

embed_W= [[ 4.71435164e-01 -1.19097569e+00  1.43270697e+00]
 [-3.12651896e-01 -7.20588733e-01  8.87162940e-01]
 [ 8.59588414e-01 -6.36523504e-01  1.56963721e-02]
 [-2.24268495e+00  1.15003572e+00  9.91946022e-01]
 [ 9.53324128e-01 -2.02125482e+00 -3.34077366e-01]
 [ 2.11836468e-03  4.05453412e-01  2.89091941e-01]
 [ 1.32115819e+00 -1.54690555e+00 -2.02646325e-01]
 [-6.55969344e-01  1.93421376e-01  5.53438911e-01]
 [ 1.31815155e+00 -4.69305285e-01  6.75554085e-01]
 [-1.81702723e+00 -1.83108540e-01  1.05896919e+00]]

id of time_emb.grads[0] 4450778768

id of time_emb.grads[0] 4450778768

